# Фаза 2 • Неделя 8 • Четверг
## Нейронные сети
### 🔥 PyTorch: fine tuning

### Подготовка к работе

1. Загрузи этот ноутбук на Google Colab
2. Подгрузи архив [датасета](https://www.kaggle.com/datasets/ikobzev/architectural-heritage-elements-image64-dataset) в свое пространство и разархивируй его с помощью `unzip`.

In [9]:
!unzip -qq /content/archive.zip

In [1]:
!pip -qq install torchutils

In [2]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms as T

# Для чтения изображений с диска
from torchvision import io # input/output
import torchutils as tu
import json
import numpy as np
import matplotlib.pyplot as plt

1. Убедись, что структура папок соответствует задаче классификации (либо приведите ее к формату, указанному ниже):

        `train`
        
            - class_1
            - class_2
            - ...
            - class_n
            
        `valid`

            - class_1
            - class_2
            - ...
            - class_n

In [14]:
!find /content -name ".ipynb_checkpoints" -type d -exec rm -r {} +

In [15]:
# transform = T.Compose([
#     T.Resize((224, 224)),
#     T.ToTensor(),
#     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])



train_transforms = T.Compose([
    T.RandomHorizontalFlip(p=0.5),         # Случайное горизонтальное отражение
    T.RandomRotation(degrees=20),          # Случайный поворот до 20 градусов
    T.RandomResizedCrop(size=224, scale=(0.8, 1.0)),  # Обрезка и изменение размера
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Изменение цвета
    T.ToTensor(),                          # Преобразование в тензор
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Нормализация
])

# Для валидации обычно используется только изменение размера и нормализация
valid_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Датасеты
train_dataset = datasets.ImageFolder(root='/content/seg_train', transform=train_transforms)
valid_dataset = datasets.ImageFolder(root='/content/seg_test', transform=valid_transforms)



2. Создай `DataLoader` в для обучающей и валидационных выборок. Примените аугментации к изображениям.

In [16]:
# Даталоадеры
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

3. Создай экземпляр предобученной модели: [torchvision models](https://pytorch.org/vision/stable/models.html). Можно взять любую модель для baseline, а дальше попробовать что-то более сложное.

In [17]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT).to(DEVICE)

fake_batch = torch.randn(4, 3, 224, 224, device=DEVICE)
tu.get_model_summary(model, fake_batch)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 189MB/s]


Layer                                       Kernel             Output          Params           FLOPs
0_conv1                                   [3, 64, 7, 7]   [4, 64, 112, 112]       9,408   472,055,808
1_bn1                                              [64]   [4, 64, 112, 112]         128    12,845,056
2_relu                                                -   [4, 64, 112, 112]           0             0
3_maxpool                                             -     [4, 64, 56, 56]           0             0
4_layer1.0.Conv2d_conv1                  [64, 64, 3, 3]     [4, 64, 56, 56]      36,864   462,422,016
5_layer1.0.BatchNorm2d_bn1                         [64]     [4, 64, 56, 56]         128     3,211,264
6_layer1.0.ReLU_relu                                  -     [4, 64, 56, 56]           0             0
7_layer1.0.Conv2d_conv2                  [64, 64, 3, 3]     [4, 64, 56, 56]      36,864   462,422,016
8_layer1.0.BatchNorm2d_bn2                         [64]     [4, 64, 56, 56]       

5. Замени выходной слой форматом, который подходит под задачу: бинарная или многоклассовая классификация.

In [36]:
model.fc = nn.Linear(512, 6)

In [24]:
for param in model.parameters():
    print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [22]:
for param in model.parameters():
    param.requires_grad = False

In [23]:
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

In [25]:
trnsfrms = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor() # автоматически интервал пикселей будет 0-1
    ]
)

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = torch.nn.CrossEntropyLoss()

6. Обучи модель (только последний слой!) и зафиксируй метрику на валидационной части выборки.

In [37]:
train_epoch_acc = []
train_epoch_losses = []
valid_epoch_losses = []
valid_epoch_acc =[]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):
    model.train()
    loss_batch = []
    acc_batch  = []

    for images, labels in train_loader:
        images = images.to(model.fc.weight.device)  # Используем device первого слоя модели для согласованности
        labels = labels.to(model.fc.weight.device)

        preds = model(images).squeeze(-1)
        loss = criterion(preds, labels.long())
        loss_batch.append(loss.item())
        accuracy = (preds.argmax(dim=1) == labels).cpu().numpy().mean()
        # accuracy = (preds.sigmoid().round() == labels).cpu().numpy().mean()
        acc_batch.append(accuracy)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_epoch_losses.append(np.mean(loss_batch))
    train_epoch_acc.append(np.mean(acc_batch))

    model.eval()
    loss_batch = []
    acc_batch  = []
    for images, labels in valid_loader:
        images = images.to(model.fc.weight.device)  # Используем device первого слоя модели для согласованности
        labels = labels.to(model.fc.weight.device)
        with torch.no_grad():
            preds = model(images).squeeze(-1)

        loss = criterion(preds, labels.long())
        loss_batch.append(loss.item())

        # accuracy = (preds.sigmoid().round() == labels).cpu().numpy().mean()
        accuracy = (preds.argmax(dim=1) == labels).cpu().numpy().mean()
        acc_batch.append(accuracy)

    valid_epoch_losses.append(np.mean(loss_batch))
    valid_epoch_acc.append(np.mean(acc_batch))

    print(f'Epoch: {epoch}  loss_train: {train_epoch_losses[-1]:.3f}, loss_valid: {valid_epoch_losses[-1]:.3f}')
    print(f'\t  metrics_train: {train_epoch_acc[-1]:.3f}, metrics_valid: {valid_epoch_acc[-1]:.3f}')

AttributeError: 'Tensor' object has no attribute 'loat'

7. Распечатай изображение из тестовой выборки и подпиши класс картинки, предсказанный моделью.

In [ ]:
# code

8. "Разморозь" несоклько слоев базовой модели и вновь обучи ее. Зафиксируй метрику качества для модели. О том, как сделать unfreeze нескольких слоев сразу, а не только последнего, можно почитать тут: [discuss.pytorch.org](https://discuss.pytorch.org/t/how-the-pytorch-freeze-network-in-some-layers-only-the-rest-of-the-training/7088/3)

In [ ]:
# code

9. Сравни качество двух моделей: предобученной с замененным выходным слоем и дообученной с несколькими размороженными слоями.

In [ ]:
# code

<img src="https://icons.iconarchive.com/icons/icons8/windows-8/256/Programming-Github-icon.png" width=32 /> Сохрани файл для __github__ и распечатай результат команды `!git status` в ячейке ниже.

In [ ]:
# code

10. Сохрани модель (пример можно посмотреть [тут](../../learning/aux/model_saving.ipynb) или в [документации](https://pytorch.org/tutorials/beginner/saving_loading_models.html))

In [ ]:
# code

11. Реализуй функцию, которая на вход принимает путь к файлу, а в ответ возвращает класс объекта.

In [ ]:
def get_prediction(path: str) -> str:
    pass

12. Сохрани ноутбук на github